In [15]:

import pandas as pd
import os
import pandas as pd
from datetime import datetime

def find_relevant_files(folder_path, text1):
    relevant_files = []
    for file_name in os.listdir(folder_path):
        if text1 in file_name:
            relevant_files.append(file_name)
    return relevant_files


folder_path = r'C:\work_folder\RFB2\Catheter_testing\Catheter_testing_Brush_F28P'
# read summary csv files
text1= '.csv'
text2= '_octiv'
text3= 'rfb'
csvfiles= find_relevant_files(folder_path, text1)


In [16]:
def parse_file_name(file_name):
    SN= file_name[0:5]
    target= "Brush" if "brush" in file_name.upper() else "F28P"
    #if "brush" in file_name:
    #    target= "Brush"
    #elif "F28P" in file_name:
    #    target= "F28P"
    return SN, target
testfile= '24560rF28P.csv'
[SN, target] = parse_file_name(testfile)
print(parse_file_name(testfile))



('24560', 'F28P')


In [17]:
def get_file_creation_date(file_path):
    creation_time = os.path.getctime(file_path)
    creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d %H:%M:%S')
    return creation_date

fp= r'C:\work_folder\RFB2\Catheter_testing\Catheter_testing_Brush_F28P\18385rbrush.csv'
dt= get_file_creation_date(fp)
print(dt)

2025-08-26 09:37:33


In [ ]:
def read_and_store_data(folder_path, text1):
    all_data = []
    relevant_files = find_relevant_files(folder_path, text1)
    
    for file_name in relevant_files:
        file_path = os.path.join(folder_path, file_name)
        temp_df = pd.read_csv(file_path)
        
        SN, target_type = parse_file_name(file_name)
        creation_date = get_file_creation_date(file_path)
        
        df['SN'] = SN
        df['creation_date'] = creation_date
        df['Target'] = target_type
        
        all_data.append(df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

folder_path = r'C:\work_folder\RFB2\Catheter_testing\Catheter_testing_Brush_F28P'
# read summary csv files
text1= '.csv'
text2= '_octiv'
text3= 'rfb'
summary_df= read_and_store_data(folder_path, text1)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 18, saw 4


In [ ]:
writepath= r'C:\work_folder\Misc_codes\prelim_gage\Data\All_data\output.csv'

In [90]:
summary_df["sample_repeat_no"].unique()

array(['SR1', '_F2'], dtype=object)

In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def create_gui(data):
    def plot_sensor_data(sensor, parse_by):
        fig, ax = plt.subplots()
        if parse_by == 'test':
            for test in data['test_no'].unique():
                test_data = data[data['test_no'] == test]
                ax.scatter( test_data['test_no'], test_data[sensor], label=f'Test {test}')
            ax.boxplot([data[data['test_no'] == test][sensor] for test in data['test_no'].unique()], positions=range(len(data['test_no'].unique())))
        elif parse_by == 'operator':
            for operator in data['operator_no'].unique():
                operator_data = data[data['operator_no'] == operator]
                ax.scatter(operator_data['operator_no'], operator_data[sensor], label=f'Operator {operator}')
            ax.boxplot([data[data['operator_no'] == operator][sensor] for operator in data['operator_no'].unique()], positions=range(len(data['operator_no'].unique())))
        
        ax.set_xlabel(parse_by)
        ax.set_ylabel(sensor)
        ax.legend()
        canvas = FigureCanvasTkAgg(fig, master=window)
        canvas.draw()
        canvas.get_tk_widget().pack()

    window = tk.Tk()
    window.title("Sensor Data Plotter")

    sensor_label = tk.Label(window, text="Select Attribute to plot:")
    sensor_label.pack()
    sensor_var = tk.StringVar()
    sensor_dropdown = ttk.Combobox(window, textvariable=sensor_var)
    sensor_dropdown['values'] = ['Ewatts', 'Awatts', 'Eff', 'PWatts', 'G3Eff', 'Real', 'Img', 'Freq']
    sensor_dropdown.pack()

    parse_by_label = tk.Label(window, text="Parse By:")
    parse_by_label.pack()
    parse_by_var = tk.StringVar()
    parse_by_dropdown = ttk.Combobox(window, textvariable=parse_by_var)
    parse_by_dropdown['values'] = ['test', 'operator']
    parse_by_dropdown.pack()

    plot_button = tk.Button(window, text="Plot", command=lambda: plot_sensor_data(sensor_var.get(), parse_by_var.get()))
    plot_button.pack()

    window.mainloop()

# Example usage
create_gui(summary_df)



In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

def create_gui2(data):
    def plot_sensor_data(sensor, parse_by, color_by):
        fig, ax = plt.subplots()
        unique_values = data[parse_by].unique()
        colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_values)))
        
        for i, value in enumerate(unique_values):
            subset = data[data[parse_by] == value]
            scatter = ax.scatter(subset[parse_by], subset[sensor], label=f'{parse_by} {value}', color=colors[i])
            ax.boxplot([subset[subset[parse_by] == value][sensor] for value in unique_values], positions=range(len(unique_values)))
        
        if color_by:
            scatter.set_array(data[color_by].astype('category').cat.codes)
            plt.colorbar(scatter, ax=ax, label=color_by)
        
        ax.set_ylabel(sensor)
        ax.set_xlabel(parse_by)
        ax.legend()
        canvas = FigureCanvasTkAgg(fig, master=window)
        canvas.draw()
        canvas.get_tk_widget().pack()

    window = tk.Tk()
    window.title("Sensor Data Plotter")

    sensor_label = tk.Label(window, text="Select Sensor:")
    sensor_label.pack()
    sensor_var = tk.StringVar()
    sensor_dropdown = ttk.Combobox(window, textvariable=sensor_var)
    sensor_dropdown['values'] = ['Ewatts', 'Awatts', 'Eff', 'PWatts', 'G3Eff', 'Real', 'Img', 'Freq']
    sensor_dropdown.pack()

    parse_by_label = tk.Label(window, text="Parse By:")
    parse_by_label.pack()
    parse_by_var = tk.StringVar()
    parse_by_dropdown = ttk.Combobox(window, textvariable=parse_by_var)
    parse_by_dropdown['values'] = ['test_no', 'operator_no', 'target_type', 'sample_repeat_no']
    parse_by_dropdown.pack()

    color_by_label = tk.Label(window, text="Color By:")
    color_by_label.pack()
    color_by_var = tk.StringVar()
    color_by_dropdown = ttk.Combobox(window, textvariable=color_by_var)
    color_by_dropdown['values'] = ['SN', 'target_type', 'operator_no', 'test_no']
    color_by_dropdown.pack()

    plot_button = tk.Button(window, text="Plot", command=lambda: plot_sensor_data(sensor_var.get(), parse_by_var.get(), color_by_var.get()))
    plot_button.pack()

    window.mainloop()




In [ ]:
summary_df= summary_df[summary_df["Eff"]>60]
create_gui2(summary_df)

In [ ]:
summary_df["SN"].unique()

In [ ]:
summary_df.head

In [ ]:
summary_df.to_csv("writepath")

In [ ]:
df1= pd.read_csv('writepath')
df1.head

In [ ]:
writepath

In [ ]:
writepath= r'C:\work_folder\Misc_codes\prelim_gage\Data\output.csv'
summary_df.to_csv(writepath)

In [ ]:
writepath

In [ ]:
df1= pd.read_csv('writepath')